# Supervision Integration with DeGirum PySDK (Hailo8L/HAILO8)

This notebook demonstrates using the Supervision library to annotate detection results from DeGirum's PySDK running on Hailo hardware. It follows the same modular style as our official user guides and covers:

- Connecting to local or cloud Hailo inference hosts
- Running object detection using YOLO models
- Visualizing predictions using `supervision` tools (bounding boxes + labels)

In [10]:
import cv2
import numpy as np
from supervision.draw.color import Color
from supervision.detection.core import Detections
from supervision.geometry.core import Position
from supervision.annotators.core import LabelAnnotator, BoxAnnotator
import degirum as dg

In [ ]:
# Inference settings
inference_host_address = "@cloud"
zoo_url = "degirum/hailo"
token = ""
device_type = ["HAILORT/HAILO8L"]
model_name = "yolov8n_relu6_face--640x640_quant_hailort_multidevice_1"
image_path = "../assets/ThreePersons.jpg"

In [ ]:
# load AI model
model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
)

print(f" Running inference using '{model_name}' on image source '{image_path}'")
inference_result = model(image_path)

In [13]:
image = cv2.imread(image_path)
if len(inference_result.results) > 0:
    bboxes = np.array([det["bbox"] for det in inference_result.results], dtype=np.float32)
    class_ids = np.array([det["category_id"] for det in inference_result.results], dtype=int)
    confidences = np.array([det["score"] for det in inference_result.results], dtype=np.float32)
    labels = [f'{det["label"]} {det["score"]:.2f}' for det in inference_result.results]

    detections = Detections(
        xyxy=bboxes,
        class_id=class_ids,
        confidence=confidences
    )
else:
    print("⚠️ No detections found.")

In [14]:
# Bounding box annotator
box_annotator = BoxAnnotator(
    color=Color(r=0, g=255, b=0),  # Green boxes
    thickness=2
)

# Label annotator with top-left alignment
label_annotator = LabelAnnotator(
    color=Color(r=0, g=0, b=0),             # Black background
    text_color=Color(r=255, g=255, b=255),  # White font
    text_position=Position.TOP_LEFT,
    text_scale=0.5,
    text_thickness=1,
    text_padding=4,
    border_radius=2
)

In [ ]:
# Apply annotations
annotated = box_annotator.annotate(scene=image.copy(), detections=detections)
annotated = label_annotator.annotate(scene=annotated, detections=detections, labels=labels)

# Show and save
cv2.imshow("Supervision Annotation (Top-Left Labels)", annotated)
print("Press 'x' or 'q' to exit.")
cv2.waitKey(0)
cv2.destroyAllWindows()